In [3]:
from pandas import read_csv, unique

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from scipy.interpolate import interp1d
from numpy import dstack
from scipy.stats import mode

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from tensorflow import stack
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling1D, BatchNormalization, MaxPool1D, Reshape, Activation
from keras.layers import Conv1D, LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [4]:
features_df = pd.read_csv("./human+activity+recognition+using+smartphones/UCI HAR Dataset/features.txt",
                          sep="\s+", header=None, names=["column_index", "column_name"])

In [5]:
feature_names = features_df["column_name"].values.tolist()

In [21]:
import pandas as pd

# 파일 경로 설정
path = "./human+activity+recognition+using+smartphones/UCI HAR Dataset/"

# 특징 파일 불러오기
X_train = pd.read_csv(path+"train/X_train.txt", sep="\s+", header=None)
X_test = pd.read_csv(path+"test/X_test.txt", sep="\s+", header=None)

# 각 특징에 대한 열 이름 추가
X_train.columns = feature_names
X_test.columns = feature_names

# X_train과 X_test를 데이터프레임으로 만들어주기
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)


In [22]:
# 레이블 파일 불러오기
y_train = pd.read_csv(path+"train/y_train.txt", sep="\s+", header=None, names=["activity"])
y_test = pd.read_csv(path+"test/y_test.txt", sep="\s+", header=None, names=["activity"])

# 클래스 레이블을 0부터 시작하도록 조정
y_train = y_train - 1
y_test = y_test - 1

In [23]:
# 훈련 데이터 생성
train_data = pd.concat([X_train, y_train], axis=1)

# 테스트 데이터 생성
test_data = pd.concat([X_test, y_test], axis=1)

In [24]:
train_data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,4
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,4
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,4
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,4
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,4


In [25]:
train_data.isna().sum()

tBodyAcc-mean()-X                       0
tBodyAcc-mean()-Y                       0
tBodyAcc-mean()-Z                       0
tBodyAcc-std()-X                        0
tBodyAcc-std()-Y                        0
                                       ..
angle(tBodyGyroJerkMean,gravityMean)    0
angle(X,gravityMean)                    0
angle(Y,gravityMean)                    0
angle(Z,gravityMean)                    0
activity                                0
Length: 562, dtype: int64

In [36]:
print(len(test_data))
test_data.head()

2947


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",activity
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,...,-0.330370,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978,4
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,...,-0.121845,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898,4
2,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,-0.939260,...,-0.190422,-0.640736,-0.034956,0.202302,0.064103,0.145068,-0.702771,0.280083,-0.079346,4
3,0.270298,-0.032614,-0.117520,-0.994743,-0.973268,-0.967091,-0.995274,-0.974471,-0.968897,-0.938610,...,-0.344418,-0.736124,-0.017067,0.154438,0.340134,0.296407,-0.698954,0.284114,-0.077108,4
4,0.274833,-0.027848,-0.129527,-0.993852,-0.967445,-0.978295,-0.994111,-0.965953,-0.977346,-0.938610,...,-0.534685,-0.846595,-0.002223,-0.040046,0.736715,-0.118545,-0.692245,0.290722,-0.073857,4


In [40]:
X_tr = train_data.drop(['activity'], axis=1)
y_tr = train_data['activity']
X_test = test_data.drop(['activity'], axis=1)
y_test = test_data['activity']
# Scale full set of predictors
X_scaled = StandardScaler().fit_transform(X_tr)

In [41]:
# Create train and test sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_tr, test_size=0.25, random_state=1, stratify = y_tr)

In [43]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape, y_test.shape

((5514, 561), (1838, 561), (5514,), (1838,), (2947, 561), (2947,))

In [44]:
# Scale train/ test predictors based on training data
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [45]:
# Reformat the training data into sequence samples
# Source: https://machinelearningmastery.com/how-to-develop-convolutional-neural-network-models-for-time-series-forecasting/
from numpy import array
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)



In [46]:
# Merge train and test X/y data to apply sequence transformation function
y_train_array = np.array(y_train)
train_set = np.c_[X_train, y_train_array]

y_test_array = np.array(y_test)
test_set = np.c_[X_test, y_test_array]

y_val_array = np.array(y_val)
val_set = np.c_[X_val, y_val_array]


# Apply sequence transformation using time step of 25 for both train and test data
# All sensing modalities are recorded at a sampling rate of 50 Hz, which is considered sufficient for capturing human activity.

X_train, y_train = split_sequences(train_set, 30)
print(X_train.shape, y_train.shape)

X_test, y_test = split_sequences(test_set, 30)
print(X_test.shape, y_test.shape)

X_val, y_val = split_sequences(val_set, 30)
print(X_val.shape, y_val.shape)

(5485, 30, 561) (5485,)
(2918, 30, 561) (2918,)
(1809, 30, 561) (1809,)


In [48]:
## reshaping data
input_shape = X_train.shape[1] * X_train.shape[2]
x_train = X_train.reshape(X_train.shape[0], input_shape)
print("Input Shape: ", input_shape)
print("Input Data Shape: ", x_train.shape)

Input Shape:  16830
Input Data Shape:  (5485, 16830)


In [47]:
# Convert output variables to categorical for CNN
y_train = to_categorical(y_train)
print(y_train.shape)

y_test = to_categorical(y_test)
print(y_test.shape)

y_val = to_categorical(y_val)
print(y_val.shape)

(5485, 6)
(2918, 6)
(1809, 6)


In [50]:
x_train = x_train.astype('float32')
y_train = y_train.astype('float32')

In [51]:
model = Sequential()
model.add(LSTM(32, return_sequences=True, input_shape=(input_shape,1), activation='relu'))
model.add(LSTM(32,return_sequences=True, activation='relu'))
model.add(Reshape((1, 240, 32)))
model.add(Conv1D(filters=64,kernel_size=2, activation='relu', strides=2))
model.add(Reshape((120, 64)))
model.add(MaxPool1D(pool_size=4, padding='same'))
model.add(Conv1D(filters=192, kernel_size=2, activation='relu', strides=1))
model.add(Reshape((29, 192)))
model.add(GlobalAveragePooling1D())
model.add(BatchNormalization(epsilon=1e-06))
model.add(Dense(6))
model.add(Activation('softmax'))

print(model.summary())

ValueError: Exception encountered when calling layer "reshape_1" (type Reshape).

total size of new array must be unchanged, input_shape = [16830, 32], output_shape = [1, 240, 32]

Call arguments received by layer "reshape_1" (type Reshape):
  • inputs=tf.Tensor(shape=(None, 16830, 32), dtype=float32)

In [52]:
from keras.models import Sequential
from keras.layers import LSTM, Reshape, Conv1D, MaxPool1D, GlobalAveragePooling1D, BatchNormalization, Dense, Activation

# 데이터 구조
input_shape = (30, 561)

model = Sequential()

# LSTM 레이어
model.add(LSTM(32, return_sequences=True, input_shape=input_shape, activation='relu'))
model.add(LSTM(32, return_sequences=True, activation='relu'))

# 1D Convolutional 레이어
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', strides=2))

# 다양한 레이어들
model.add(Reshape((1, 120, 64)))  # Reshape 추가
model.add(MaxPool1D(pool_size=4, padding='same'))
model.add(Conv1D(filters=192, kernel_size=2, activation='relu', strides=1))
model.add(Reshape((29, 192)))
model.add(GlobalAveragePooling1D())
model.add(BatchNormalization(epsilon=1e-06))

# Fully Connected 레이어
model.add(Dense(6))
model.add(Activation('softmax'))

print(model.summary())


ValueError: Exception encountered when calling layer "reshape_2" (type Reshape).

total size of new array must be unchanged, input_shape = [15, 64], output_shape = [1, 120, 64]

Call arguments received by layer "reshape_2" (type Reshape):
  • inputs=tf.Tensor(shape=(None, 15, 64), dtype=float32)

In [53]:
from scipy.stats import mode

def segments(df, time_steps, step):
    N_FEATURES = df.shape[1] - 1  # Assume all columns in the DataFrame are features
    segments = []
    labels = []
    
    # Assuming the last column is the label (activity)
    label_name = df.columns[-1]  
    
    for i in range(0, len(df) - time_steps, step):
        segment = df.iloc[i:i+time_steps, :-1]  # Exclude the last column (label)
        label = mode(df[label_name][i:i+time_steps])[0][0]
        
        segments.append(segment.values.T)  # Transpose to get [X, Y, Z]
        labels.append(label)
    
    reshaped_segments = np.asarray(segments, dtype=np.float32).reshape(-1, time_steps, N_FEATURES)
    labels = np.asarray(labels)
    
    return reshaped_segments, labels


In [ ]:
TIME_PERIOD = 80
STEP_DISTANCE = 40
x_train, y_train = segments(df_train, TIME_PERIOD, STEP_DISTANCE)